In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
!kaggle competitions download -c UBC-OCEAN

100%|██████████████████████████████████████▉| 719G/719G [1:21:33<00:00, 145MB/s]
100%|███████████████████████████████████████| 719G/719G [1:21:33<00:00, 158MB/s]


In [2]:
!unzip UBC-OCEAN -d data > /dev/null

In [ ]:
import os
import pandas as pd
from PIL import Image
import concurrent.futures

Image.MAX_IMAGE_PIXELS = None

# Create the 'tiles' directory if it doesn't exist
if not os.path.exists('tiles'):
    os.makedirs('tiles')

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/train.csv')

# Function to split an image into tiles and save them
def process_image(row):
    image_id, height, width = row['image_id'], row['image_height'], row['image_width']
    print(f"processing image {image_id}")
    image_path = f'data/train_images/{image_id}.png'
    
    # Create the image directory if it doesn't exist
    if not os.path.exists(f'tiles/{image_id}'):
        os.makedirs(f'tiles/{image_id}')
    
    with Image.open(image_path) as image:
        for i in range(0, height, 224):
            for j in range(0, width, 224):
                tile = image.crop((j, i, j + 224, i + 224))
                # Make sure the tile size is 224x224
                if tile.size[0] != 224 or tile.size[1] != 224:
                    continue
                # Check if the tile is more than 90% black/white
                black_or_white_pixels = sum(1 for pixel in tile.getdata() if sum(pixel) < 50 or sum(pixel) > 255 * 3 - 50)
                if black_or_white_pixels / (224 * 224) >= 0.9:
                    continue

                tile.save(f'tiles/{image_id}/{i}_{j}.png')

# Function to parallelize the image processing
def parallel_process_images(data_frame):
    # Using a ThreadPoolExecutor to create and manage a pool of threads
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        # Submitting all image processing tasks to the thread pool
        futures = [executor.submit(process_image, row) for _, row in data_frame.iterrows()]
        
        # Waiting for all futures to complete and gathering the results
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    return results

# Process each image in parallel
parallel_process_images(df)


processing image 3191processing image 5851processing image 2906processing image 6281



processing image 6363
processing image 6898
processing image 8279
processing image 8713
processing image 9183
processing image 9254


In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Load your data
train = pd.read_csv("data/train.csv")

# Create a column that combines 'is_tma' and 'label' for stratification
train['stratify_col'] = train['is_tma'].astype(str) + "_" + train['label'].astype(str)

# Determine the smallest group size for stratification
min_group_size = train['stratify_col'].value_counts().min()

# Determine the maximum number of folds
max_k = min_group_size // 1  # Each group needs at least 1 sample

# Initialize KFold
skf = StratifiedKFold(n_splits=max_k)

# Create folds
for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['stratify_col'])):
    train_fold = train.iloc[train_idx]
    val_fold = train.iloc[val_idx]

    # Remove the 'stratify_col' before saving
    train_fold = train_fold.drop(columns=['stratify_col'])
    val_fold = val_fold.drop(columns=['stratify_col'])

    # Save each fold to CSV
    train_fold.to_csv(f"folds/train_fold_{fold}.csv", index=False)
    val_fold.to_csv(f"folds/val_fold_{fold}.csv", index=False)


In [1]:
import os
import pandas as pd
from PIL import Image
import concurrent.futures
import numpy as np

Image.MAX_IMAGE_PIXELS = None
IMAGE_SIZE = 2048
RESIZE_SIZE = 1024

# Create the 'tiles' directory if it doesn't exist
if not os.path.exists(f'tiles_{IMAGE_SIZE}'):
    os.makedirs(f'tiles_{IMAGE_SIZE}')

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/train.csv')

def calculate_entropy(tile):
    # Convert to grayscale if the image is RGB
    if tile.mode == 'RGB':
        tile = tile.convert('L')

    # Flatten the tile and calculate histogram
    pixel_counts = np.histogram(tile, bins=range(256))[0]

    # Normalize to get probabilities
    probabilities = pixel_counts / np.sum(pixel_counts)

    # Filter out zero probabilities and calculate entropy
    probabilities = probabilities[probabilities > 0]
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

# Function to split an image into tiles and save them
def process_image(row):
    image_id, height, width = row['image_id'], row['image_height'], row['image_width']
    print(f"processing image {image_id}")
    image_path = f'data/train_images/{image_id}.png'
    
    # Create the image directory if it doesn't exist
    if not os.path.exists(f'tiles_{IMAGE_SIZE}/{image_id}'):
        os.makedirs(f'tiles_{IMAGE_SIZE}/{image_id}')
   
    with Image.open(image_path) as image:
        for i in range(0, height, IMAGE_SIZE):
            for j in range(0, width, IMAGE_SIZE):
                tile = image.crop((j, i, j + IMAGE_SIZE, i + IMAGE_SIZE))
                # Make sure the tile size is IMAGE_SIZExIMAGE_SIZE and there's some data there
                if tile.size[0] != IMAGE_SIZE or tile.size[1] != IMAGE_SIZE or calculate_entropy(tile) < 3:
                    continue
                tile = tile.resize((RESIZE_SIZE, RESIZE_SIZE), Image.Resampling.LANCZOS)

                tile.save(f'tiles_{IMAGE_SIZE}/{image_id}/{i}_{j}.png')

# Function to parallelize the image processing
def parallel_process_images(data_frame):
    # Using a ThreadPoolExecutor to create and manage a pool of threads
    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
        # Submitting all image processing tasks to the thread pool
        futures = [executor.submit(process_image, row) for _, row in data_frame.iterrows()]
        
        # Waiting for all futures to complete and gathering the results
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    return results

# Process each image in parallel
parallel_process_images(df)

# for _, row in df.iterrows():
#     process_image(row)


processing image 66processing image 4

processing image 91
processing image 281
processing image 286
processing image 431
processing image 706
processing image 970
processing image 1020
processing image 1080
processing image 1101
processing image 1252
processing image 1289
processing image 1295
processing image 1660
processing image 1666
processing image 1774
processing image 1925
processing image 1943
processing image 1952
processing image 2097
processing image 2227
processing image 2391
processing image 2666
processing image 2706
processing image 2906
processing image 3055
processing image 3084
processing image 3092
processing image 3098
processing image 3191
processing image 3222
processing image 3264
processing image 3511
processing image 3672
processing image 3881
processing image 3997
processing image 4134
processing image 4211
processing image 4608
processing image 4797
processing image 4827
processing image 4877
processing image 4963
processing image 5015
processing image 5114


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [5]:
import pandas as pd
from sklearn.utils import shuffle

# Load the training data
train = pd.read_csv("data/train.csv")

train = train[train['is_tma'] == False]
# Shuffle the data
train_shuffled = shuffle(train, random_state=42)

# Step 1: Group by the 'label' column
grouped = train_shuffled.groupby('label')

# Step 2: Sample 5 instances from each group
validation_set = grouped.apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

# Step 3: Find the remaining training data
train_not_validation = train_shuffled[~train_shuffled['image_id'].isin(validation_set['image_id'])]

# Step 4: Save to CSV files
validation_set.to_csv("validation-no-tma.csv", index=False)
train_not_validation.to_csv("train-no-tma.csv", index=False)


In [6]:
import pandas as pd
from sklearn.utils import shuffle

# Load the training data
train = pd.read_csv("data/train.csv")

train = train[train['is_tma'] == True]
# Shuffle the data
train_shuffled = shuffle(train, random_state=42)

# Step 1: Group by the 'label' column
grouped = train_shuffled.groupby('label')

# Step 2: Sample 5 instances from each group
validation_set = grouped.apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

# Step 3: Find the remaining training data
train_not_validation = train_shuffled[~train_shuffled['image_id'].isin(validation_set['image_id'])]

# Step 4: Save to CSV files
validation_set.to_csv("validation-yes-tma.csv", index=False)
train_not_validation.to_csv("train-yes-tma.csv", index=False)


In [32]:
import os
import pandas as pd
from PIL import Image
import concurrent.futures

Image.MAX_IMAGE_PIXELS = None

# Create the 'tiles' directory if it doesn't exist
if not os.path.exists('tiles'):
    os.makedirs('tiles')

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('data/train.csv')

# Function to split an image into tiles, scale down, and save them
def process_image(row):
    image_id, height, width = row['image_id'], row['image_height'], row['image_width']
    print(f"processing image {image_id}")
    image_path = f'data/train_images/{image_id}.png'
    
    # Create the image directory if it doesn't exist
    image_dir = f'tiles_2964/{image_id}'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    
    with Image.open(image_path) as image:
        for i in range(0, height, 2964):
            for j in range(0, width, 2964):
                # Make sure we don't go out of image bounds
                tile = image.crop((j, i, min(j + 2964, width), min(i + 2964, height)))
                
                # Skip any tiles that are not full-size (e.g., at edges)
                if tile.size[0] != 2964 or tile.size[1] != 2964:
                    continue
                
                # Resize tile to 224x224
                tile = tile.resize((224, 224), Image.Resampling.LANCZOS)

                # Check if the tile is more than 90% black/white
                black_or_white_pixels = sum(1 for pixel in tile.getdata() if sum(pixel) < 50 or sum(pixel) > 255 * 3 - 50)
                if black_or_white_pixels / (224 * 224) >= 0.9:
                    continue

                # Save the tile
                tile.save(f'{image_dir}/{i}_{j}.png')

# # Function to parallelize the image processing
# def parallel_process_images(data_frame):
#     # Using a ThreadPoolExecutor to create and manage a pool of threads
#     with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
#         # Submitting all image processing tasks to the thread pool
#         futures = [executor.submit(process_image, row) for _, row in data_frame.iterrows()]
        
#         # Waiting for all futures to complete and gathering the results
#         results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
#     return results

# Process each image in parallel
# parallel_process_images(df)

for _, row in df.iterrows():
    process_image(row)

processing image 2906
processing image 3191
processing image 5851
processing image 6281
processing image 6363
processing image 6898
processing image 8279
processing image 8713
processing image 9183
processing image 9254
processing image 10252
processing image 11263
processing image 14051
processing image 14401
processing image 14617
processing image 15231
processing image 16209
processing image 22740
processing image 25331
processing image 27739
processing image 29147
processing image 29904
processing image 32035
processing image 34649
processing image 34690
processing image 34822
processing image 36063
processing image 36499
processing image 38366
processing image 42549
processing image 44232
processing image 44530
processing image 48506
processing image 48550
processing image 49587
processing image 49872
processing image 50962
processing image 51032
processing image 51128
processing image 53377
processing image 54007
processing image 56117
processing image 56351
processing image 5694

In [7]:
import json
import subprocess
import os
import zipfile

with open('data/updated_image_ids.json', 'r') as json_file:
    file_list = json.load(json_file)

competition_name = 'UBC-OCEAN'

for file_name in file_list:
    file_path = f'data/train_images/{file_name}.png'
    # Check if the file already exists and delete it
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f'Existing file {file_name}.png deleted.')

    # Execute the Kaggle command to download the file
    command = f'kaggle competitions download -c {competition_name} -f train_images/{file_name}.png -p data/train_images'
    try:
        subprocess.run(command, shell=True, check=True)
        
        downloaded_file_path = f'data/train_images/{file_name}.png.zip'
        # Check if the downloaded file is a zip file
        if os.path.exists(downloaded_file_path):
            with zipfile.ZipFile(downloaded_file_path, 'r') as zip_ref:
                zip_ref.extractall('data/train_images')
            print(f'Successfully unzipped {file_name}.png.zip')
            os.remove(downloaded_file_path)
            print(f'Deleted the zip file: {file_name}.png.zip')
        else:
            print(f'Successfully downloaded {file_name}.png')

    except subprocess.CalledProcessError as e:
        print(f'Error downloading {file_name}.png: {e}')

print('Download process completed.')


Existing file 2906.png deleted.


100%|██████████| 1.62G/1.62G [00:14<00:00, 119MB/s] 



Successfully unzipped 2906.png.zip
Deleted the zip file: 2906.png.zip
Existing file 3191.png deleted.


100%|██████████| 1.06G/1.06G [00:13<00:00, 83.3MB/s]



Successfully downloaded 3191.png
Existing file 5851.png deleted.


...


Successfully unzipped 62476.png.zip
Deleted the zip file: 62476.png.zip
Existing file 63165.png deleted.


100%|██████████| 1.17G/1.17G [00:11<00:00, 108MB/s] 



Successfully unzipped 63165.png.zip
Deleted the zip file: 63165.png.zip
Existing file 64111.png deleted.


100%|██████████| 594M/594M [00:06<00:00, 97.5MB/s] 



Successfully unzipped 64111.png.zip
Deleted the zip file: 64111.png.zip
Download process completed.


In [1]:
!kaggle datasets download -d sohier/ubc-ovarian-cancer-competition-supplemental-masks

 95%|█████████████████████████████████████▉  | 337M/356M [00:08<00:00, 22.7MB/s]
100%|████████████████████████████████████████| 356M/356M [00:08<00:00, 43.8MB/s]


In [2]:
!unzip -q ubc-ovarian-cancer-competition-supplemental-masks.zip -d data/train_mask_images